In [ ]:
import os

import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy

exp_code = "azide_v2"

# specify correct parent folder to raw data here
PROJECT_PATH = os.path.join(
    r"C:\Users\rober\Downloads\raw spectra\raw spectra\IR", exp_code
)

DELIMITER = ","


def lambert_beer(
    intensity_blank: list[float], intensity_sample: list[float]
) -> np.ndarray:
    """A = log(T_0 / T_t)"""
    return np.log10(intensity_blank / intensity_sample)
    return gaussian_filter1d(np.log10(intensity_blank / intensity_sample), sigma=3)

In [ ]:
plt.rcParams.update(
    {
        "font.size": 24,
        "axes.linewidth": 3,
        "xtick.major.width": 3,
        "ytick.major.width": 3,
        "xtick.minor.width": 3,
        "ytick.minor.width": 3,
    }
)

In [ ]:
def find_peak(
    wavelengths: pd.Series, intensities: pd.Series, peak: float, width: float
) -> tuple[int, float, float]:
    if len(wavelengths) != len(intensities):
        raise ValueError("x_values and intensity must have the same length")
    selector = (wavelengths >= peak - width) & (wavelengths <= peak + width)
    pos_intensities = abs(intensities[selector])
    index_of_max_intensity = pos_intensities[selector].idxmax()
    max_wavelength = wavelengths[selector][index_of_max_intensity]
    max_intensity = intensities[selector][index_of_max_intensity]
    return index_of_max_intensity, max_wavelength, max_intensity


def integrate_area(
    wavelengths: list[float], intensities: list[float], upper: float, lower: float
) -> float:
    if len(wavelengths) != len(intensities):
        raise ValueError("x_values and intensity must have the same length")
    selector = (wavelengths >= lower) & (wavelengths <= upper)
    return np.trapz(wavelengths[selector], intensities[selector])


def integrate_peak(
    wavelengths: list[float], intensities: list[float], peak: float, width: float
) -> float:
    if len(wavelengths) != len(intensities):
        raise ValueError("x_values and intensity must have the same length")
    selector = (wavelengths >= peak - width) & (wavelengths <= peak + width)
    return np.trapz(intensities[selector], wavelengths[selector])

In [ ]:
from datetime import datetime


MIN_WAVELENGTH = 1550
MAX_WAVELENGTH = 4000

PEAK_1 = 2097
PEAK_2 = 2173
PEAK_WIDTH = 10

CUT_START = 2600
CUT_END = 3100

start_timestamp = None
times = []
data = []
file_names: list[str] = []
common_x_axis = None

peak_1_wavelength = []
peak_2_wavelength = []
peak_1_intensity = []
peak_2_intensity = []
peak_1_area = []
peak_2_area = []

for file_name in os.listdir(PROJECT_PATH):
    file_path = os.path.join(PROJECT_PATH, file_name)
    if (
        os.path.isfile(file_path)
        and file_name.endswith(".csv")
        and not "fig_raw_data" in file_name
    ):
        file_names.append(file_path)

for i, file_name in tqdm(enumerate(file_names)):
    timestamp = datetime.strptime(
        file_name.split("\\")[-1].removesuffix(".csv"), "-%Y-%m-%d-%H-%M-%S"
    )
    if start_timestamp is None:
        start_timestamp = timestamp
    times.append((timestamp - start_timestamp).total_seconds() / 60)

    df_curr_spectrum = pd.read_csv(
        file_names[i], names=["wavelength", "intensity"], delimiter=DELIMITER
    )
    df_curr_spectrum = df_curr_spectrum[
        df_curr_spectrum["wavelength"].between(
            MIN_WAVELENGTH, MAX_WAVELENGTH, inclusive="both"
        )
    ]

    # cut out noisy region
    df_curr_spectrum = df_curr_spectrum[
        ~df_curr_spectrum["wavelength"].between(CUT_START, CUT_END, inclusive="both")
    ]
    df_curr_spectrum = df_curr_spectrum.reset_index(drop=True)

    if common_x_axis is None:
        common_x_axis = df_curr_spectrum["wavelength"]
    else:
        if not common_x_axis.equals(df_curr_spectrum["wavelength"]):
            print(f"File {file_name} has different x axis")
            continue

    data.append(df_curr_spectrum["intensity"])

    peak_1 = find_peak(
        df_curr_spectrum["wavelength"],
        df_curr_spectrum["intensity"],
        PEAK_1,
        PEAK_WIDTH,
    )
    peak_2 = find_peak(
        df_curr_spectrum["wavelength"],
        df_curr_spectrum["intensity"],
        PEAK_2,
        PEAK_WIDTH,
    )

    peak_1_wavelength.append(peak_1[1])
    peak_2_wavelength.append(peak_2[1])

    peak_1_intensity.append(peak_1[2])
    peak_2_intensity.append(peak_2[2])

    peak_1_area.append(
        integrate_peak(
            df_curr_spectrum["wavelength"],
            df_curr_spectrum["intensity"],
            peak_1[1],
            PEAK_WIDTH,
        )
    )
    peak_2_area.append(
        integrate_peak(
            df_curr_spectrum["wavelength"],
            df_curr_spectrum["intensity"],
            peak_2[1],
            PEAK_WIDTH,
        )
    )

In [ ]:
# average peak positions
average_peak_1 = np.mean(peak_1_wavelength[4:70])
average_peak_2 = np.mean(peak_2_wavelength[4:70])
print(f"Average peak 1: {average_peak_1}")
print(f"Average peak 2: {average_peak_2}")

In [ ]:
peak_1_area_fixed = []
peak_2_area_fixed = []

for spectrum in data:
    peak_1_area_fixed.append(
        integrate_peak(common_x_axis, spectrum, average_peak_1, PEAK_WIDTH)
    )
    peak_2_area_fixed.append(
        integrate_peak(common_x_axis, spectrum, average_peak_2, PEAK_WIDTH)
    )

In [ ]:
plt.plot(times, peak_1_intensity, label=f"{peak_1[1]:.0f} cm^-1")
plt.plot(times, peak_2_intensity, label=f"{peak_2[1]:.0f} cm^-1")
plt.xlabel("Time / min")
plt.ylabel("Intensity")
plt.title("Single peak intensity")
plt.legend()
# plt.savefig(f"{exp_code}_peak_intensity.svg", transparent=True)

In [ ]:
plt.plot(
    times,
    peak_1_area_fixed,
    label=f"{average_peak_1:.0f} cm^-1",
    marker="o",
    linestyle="None",
)
plt.plot(
    times,
    peak_2_area_fixed,
    label=f"{average_peak_2:.0f} cm^-1",
    marker="o",
    linestyle="None",
)

df = pd.DataFrame(
    data={"time": times, "peak_1": peak_1_area_fixed, "peak_2": peak_2_area_fixed}
)
window = 10
df["peak_1_smooth"] = df["peak_1"].rolling(window=window).mean()
df["peak_2_smooth"] = df["peak_2"].rolling(window=window).mean()
df["peak_1_std"] = df["peak_1"].rolling(window=window).std()
df["peak_2_std"] = df["peak_2"].rolling(window=window).std()

# 95 % confidence interval
ci_1 = 1.96 * df["peak_1_std"] / np.sqrt(window)
ci_2 = 1.96 * df["peak_2_std"] / np.sqrt(window)

plt.fill_between(
    df["time"], df["peak_1_smooth"] - ci_1, df["peak_1_smooth"] + ci_1, alpha=0.5
)
plt.fill_between(
    df["time"], df["peak_2_smooth"] - ci_2, df["peak_2_smooth"] + ci_2, alpha=0.5
)

# plot the rolling average in the same color as the original data
plt.plot(df["time"], df["peak_1_smooth"], color="tab:blue")
plt.plot(df["time"], df["peak_2_smooth"], color="tab:orange")

plt.xlabel("Time / min")
plt.ylabel("Area")
plt.title("Single peak areas")
plt.legend().get_frame().set_linewidth(3)
# plt.savefig(f"{exp_code}_peak_area.svg", transparent=True)

In [ ]:
from scipy.stats import linregress


def detect_plateau_from_slope(
    values, num_datapoints: int = 5, threshold: float = 1e-4
) -> int:
    """
    Detects a plateau in a list of values by calculating the slope of a linear regression
    for the last num_datapoints and comparing it to a threshold value.
    """
    for i in range(len(values) - num_datapoints):
        if i < num_datapoints:
            continue
        slope = linregress(
            range(num_datapoints),
            np.array(values[i - num_datapoints : i]) / max(values),
        ).slope
        if abs(slope) < threshold:
            print(f"Reaction has reached plateau after {i} datapoints.")
            return i
    return len(values)

In [ ]:
smoothed_data = deepcopy(data)
for i in range(len(smoothed_data)):
    smoothed_data[i] = gaussian_filter1d(smoothed_data[i], sigma=10)

In [ ]:
from scipy.integrate import trapezoid

error_tolerance = 1e-3
spectra_array = data


# Calculate Jaccard index for pair of spectra
def jaccard_next_two_spectra(index_1, index_2, reference_spectrum=None):
    if reference_spectrum is None:
        spectrum1_y = spectra_array[index_1]
    else:
        spectrum1_y = reference_spectrum
    spectrum2_y = spectra_array[index_2]
    union, intersection = [], []
    for y1, y2 in zip(spectrum1_y, spectrum2_y):
        if abs(y1 - y2) < error_tolerance:
            union.append(y2)
            intersection.append(y2)
        else:
            union.append(max(y1, y2))
            intersection.append(min(y1, y2))
    intersection_area = trapezoid(intersection, common_x_axis)
    union_area = trapezoid(union, common_x_axis)
    return intersection_area / union_area


print("Calculating Jaccard indices...")
jaccards = []
for i in tqdm(range(len(spectra_array))):
    jaccards.append(jaccard_next_two_spectra(8, i))

print("Calculating reverse Jaccard indices...")
reverse_jaccards = []
for i in tqdm(range(len(spectra_array))):
    reverse_jaccards.append(jaccard_next_two_spectra(i, len(spectra_array) - 1))

print("Calclating neighbor Jaccard indices...")
neighbor_jaccards = []
for i in tqdm(range(len(spectra_array) - 1)):
    neighbor_jaccards.append(jaccard_next_two_spectra(i, i + 1))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plateau_index = detect_plateau_from_slope(jaccards, num_datapoints=5, threshold=1e-3)
print(f"Plateau time: {times[plateau_index]} min")

plt.plot(times, jaccards, marker="o", linestyle="None", label=f"Jaccard index")

# plot 95 % confidence interval of moving average
window = 5
df = pd.DataFrame(data={"time": times, "jaccard": jaccards})
df["jaccard_smooth"] = df["jaccard"].rolling(window=window).mean()
df["jaccard_std"] = df["jaccard"].rolling(window=window).std()

# 95 % confidence interval
ci = 1.96 * df["jaccard_std"] / np.sqrt(window)
plt.fill_between(
    df["time"], df["jaccard_smooth"] - ci, df["jaccard_smooth"] + ci, alpha=0.5
)

# plot the rolling average
plt.plot(
    times, df["jaccard_smooth"], label=f"moving average ({window} points)", color="C0"
)

# plot vertical line at plateau index
plt.axvline(
    x=times[plateau_index],
    color="r",
    linestyle="--",
    label=f"Plateau after {times[plateau_index]:.2f} min",
)
# plt.xticks(rotation=45)
plt.legend().get_frame().set_linewidth(3)
plt.xlabel("Time / min")
plt.ylabel("Jaccard index")
# plt.title(f'Jaccard index for {exp_code}')
plt.savefig(f"{exp_code}_jaccard.svg", transparent=True, bbox_inches="tight")
plt.show()

Kinetics

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

plt.plot(
    times,
    peak_1_area_fixed,
    label=f"{average_peak_1:.0f} cm^-1",
    marker="o",
    linestyle="None",
)
plt.plot(
    times,
    peak_2_area_fixed,
    label=f"{average_peak_2:.0f} cm^-1",
    marker="o",
    linestyle="None",
)

times = np.array(times)


# fit exponential function for kinetic first order to the first peak
def exp_func(t, a, b, c):
    return -a * np.exp(-b * t) + c


# fit difference of exponentials to second peak (formation and decay of intermediate)
def diff_exp_func(t, a, b, c, d, e):
    return a * np.exp(-b * t) - c * np.exp(-d * t) + e


# alternatively fit a combination of linear and exponential functions
def lin_exp_func(t, a, b, c):
    return a * t * np.exp(-b * t) + c


from scipy.optimize import curve_fit

popt, pcov = curve_fit(exp_func, times, np.array(peak_1_area_fixed), p0=[40, 0.1, 30])
print(f"Optimal parameters: {popt}")
plt.plot(times, exp_func(times, *popt), color="tab:blue")

popt, pcov = curve_fit(
    lin_exp_func, times, np.array(peak_2_area_fixed), p0=[30, 0.1, 0.4]
)
print(f"Optimal parameters: {popt}")
plt.plot(times, lin_exp_func(times, *popt), color="tab:orange")


plt.xlabel("Time / min")
plt.ylabel("Area")
# plt.title("Single peak areas")
plt.legend().get_frame().set_linewidth(3)
plt.savefig(f"{exp_code}_peak_area.svg", transparent=True, bbox_inches="tight")